In [1]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit


file = open("../log/(1).pickle", "rb")
data = pickle.load(file)
file.close()

# print(data)

In [2]:
game_info = data['ml']['scene_info']

game_command = data['ml']['command']

# for i in range(2, 48):
#     path = "../log/lin ("+str(i)+").pickle"
#     file = open(path, "rb")
#     data = pickle.load(file)
#     game_info = game_info + data['ml']['scene_info']
#     game_command = game_command + data['ml']['command']
#     file.close()

# file = open("../log/(2).pickle", "rb")
# data = pickle.load(file)
# game_info = game_info + data['ml']['scene_info']
# game_command = game_command + data['ml']['command']
# file.close()
    
print(len(game_info))
print(len(game_command))

221396
221396


In [3]:

g = game_info[1]

feature = np.array([g['snake_head'][0], g['snake_head'][1]])
# feature = np.array([g['snake_head'][0], g['snake_head'][1], g['food'][0], g['food'][1]])
game_command[1] = 0
# print(len(game_info))
for i in range(2, len(game_info) - 1):
    g = game_info[i]
    feature = np.vstack((feature, [[g['snake_head'][0], g['snake_head'][1]]]))
#     feature = np.vstack((feature, [[g['snake_head'][0], g['snake_head'][1], g['food'][0], g['food'][1]]]))
    if game_command[i] == "NONE": game_command[i] = 0
    elif game_command[i] == "UP": game_command[i] = 1
    elif game_command[i] == "DOWN": game_command[i] = 2
    elif game_command[i] == "LEFT": game_command[i] = 3
    else: game_command[i] = 4
    
answer = np.array(game_command[1:-1])

print(feature)
print(feature.shape)
print(answer)
print(answer.shape)

[[ 50  40]
 [ 60  40]
 [ 70  40]
 ...
 [ 60 270]
 [ 60 280]
 [ 60 290]]
(221394, 2)
[0 4 4 ... 4 4 4]
(221394,)


In [4]:
#資料劃分
x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=1/3, random_state=85)
#參數區間
param_grid = {'n_neighbors':[1, 2, 3]}
#交叉驗證 
cv = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=12)
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=cv, verbose=10, n_jobs=-1) #n_jobs為平行運算的數量
grid.fit(x_train, y_train)
grid_predictions = grid.predict(x_test)

#儲存
file = open('model0610.pickle', 'wb')
pickle.dump(grid, file)
file.close()

Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:    6.3s remaining:   12.6s
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    6.3s remaining:    6.3s
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    6.3s remaining:    3.1s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    6.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    6.3s finished


In [5]:
#最佳參數
print(grid.best_params_)
#預測結果
#print(grid_predictions)
#混淆矩陣
print(confusion_matrix(y_test, grid_predictions))
#分類結果
print(classification_report(y_test, grid_predictions))

{'n_neighbors': 2}
[[    0     1     0     0     0]
 [    0 14170   570  2106  1518]
 [    0  1724 12869  1980  1892]
 [    0  6033  6090  5684   646]
 [    0  6695  6288  1556  3976]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      0.77      0.60     18364
           2       0.50      0.70      0.58     18465
           3       0.50      0.31      0.38     18453
           4       0.50      0.21      0.30     18515

    accuracy                           0.50     73798
   macro avg       0.40      0.40      0.37     73798
weighted avg       0.50      0.50      0.47     73798



C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
